In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import os.path as osp
import cv2
from vehicle_reid.embedder import Embedder
from vehicle_reid.evaluate import Evaluator
from vehicle_reid import exp_config
from vehicle_reid.common import *
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import h5py
EXP_ROOT = "/home/hthieu/AICityChallenge2019/track2_experiments/"
GPU_ID = "0"
query_cfg = QUERY
test_full_cfg = TESTFULL
train_full_cfg = TRAINFULL

In [6]:
#EMBED WHEEL
embedder = Embedder(exp_config.WheelEmbedder.EXP_ID,GPU_ID)
embedder.embed_csv_file(QUERY_WHEEL['root'], QUERY_WHEEL['csv_in'], QUERY_WHEEL["out_h5"])
embedder.embed_csv_file(TEST_WHEEL['root'], TEST_WHEEL['csv_in'], TEST_WHEEL["out_h5"])

Loading args from /home/hthieu/AICityChallenge2019/track2_experiments/100519_triplet-reid_wheel/args.json.
False False
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Save h5 file to:  query_ext/wheel_query.h5
Restoring from checkpoint: /home/hthieu/AICityChallenge2019/track2_experiments/100519_triplet-reid_wheel/checkpoint-18080
INFO:tensorflow:Restoring parameters from /home/hthieu/AICityChallenge2019/track2_experiments/100519_triplet-reid_wheel/checkpoint-18080
Embedded batch 896-983/983
Done with embedding, aggregating augmentations...
(983, 512)
(983, 512)
False False
INFO:tensorflow:Scale of 0 disables regularizer.
Save h5 file to:  query_ext/wheel_test.h5
Restoring from checkpoint: /home/hthieu/AICityChallenge2019/track2_experiments/100519_triplet-reid_wheel/checkpoint-18080
INFO:tensorflow:Restoring parameters from /home/hthieu/AICityChallenge2019/track2_experiments/100519_triplet-reid_wheel/checkpoint-18

In [ ]:
embedder = Embedder(exp_config.Track2Embedder.EXP_ID,GPU_ID)
embedder.embed_csv_file(TRAIN_BUS['root'], TRAIN_BUS['csv_in'], TRAIN_BUS["out_h5"], flip = True, crop = 'five')

In [ ]:
#EMBED QUERY
embedder = Embedder(exp_config.Track2Embedder.EXP_ID,GPU_ID)
embedder.embed_csv_file(query_cfg['root'], query_cfg['csv_in'], query_cfg["out_h5"])

In [ ]:
#EMBED TEST
embedder = Embedder(exp_config.Track2Embedder.EXP_ID,GPU_ID)
embedder.embed_csv_file(test_full_cfg['root'], test_full_cfg['csv_in'], test_full_cfg["out_h5"])

In [ ]:
#EMBED TRAIN
embedder = Embedder(exp_config.Track2Embedder.EXP_ID,GPU_ID)
embedder.embed_csv_file("/home/hthieu/AICityChallenge2019/data/", train_full_cfg['csv_in'], train_full_cfg["out_h5"])

In [ ]:
#READ EMB BUS RESULTS
train_emb, train_id, train_track = load_emb_h5(train_cfg['out_h5'])
test_emb, test_id, test_track = load_emb_h5(test_cfg['out_h5'])
test_full_emb, test_full_id, test_full_track = load_emb_h5(test_full_cfg['out_h5'])
train_full_emb, train_full_id, train_full_track = load_emb_h5(train_full_cfg['out_h5'])
query_emb, _, __ = load_emb_h5(query_cfg['out_h5'])

In [ ]:
train_ids, train_paths, train_track = read_csv_dataset(train_cfg['csv_in'])
test_ids, test_paths, test_track = read_csv_dataset(test_cfg['csv_in'])
_, query_paths, __ = read_csv_dataset(query_cfg['csv_in'])
test_full_ids, test_full_paths, test_full_track = read_csv_dataset(test_full_cfg['csv_in'])

train_uni, truid = np.unique(train_track, return_index=True)
test_uni,teuid = np.unique(test_track,  return_index=True)
test_full_uni, tefuid = np.unique(test_full_track,  return_index=True)
dict_train3imgs = get3avt("train")
dict_test3imgs = get3avt("test")
train3 = np.array([np.argwhere(np.array(train_paths) == np.array(dict_train3imgs[x])[:,None])[:,1] for x in train_uni])

test3  = np.array([np.argwhere(np.array(test_paths) == np.array(dict_test3imgs[x])[:,None])[:,1] for x in test_uni])

train_paths = np.array([osp.join(train_cfg['root'], x) for x in train_paths])
test_paths = np.array( [osp.join(test_cfg['root'], x) for x in test_paths])
query_paths = np.array( [osp.join(query_cfg['root'], x) for x in query_paths])
test_full_paths = np.array( [osp.join(test_full_cfg['root'], x) for x in test_full_paths])
train_uni_id = train_ids[np.array(truid)]

In [ ]:
#USE 3 AVT
dict_train3imgs = get3avt("train")
dict_test3imgs = get3avt("test")
train3 = np.array([np.argwhere(np.array(train_paths) == np.array(dict_train3imgs[x])[:,None])[:,1] for x in train_uni])
test3  = np.array([np.argwhere(np.array(test_paths) == np.array(dict_test3imgs[x])[:,None])[:,1] for x in test_uni])
train_track_emb = np.average(train_emb[train3],axis = 1)
test_track_emb = np.average(test_emb[test3],axis = 1)

In [ ]:
#USE ALL
train_track_emb = np.array([np.average(train_emb[train_track == x], axis = 0) for x in train_uni])
test_track_emb = np.array([np.average(test_emb[test_track == x], axis = 0) for x in test_uni])
test_full_track_emb = np.array([np.average(test_full_emb[test_full_track == x], axis = 0) for x in test_full_uni])

In [ ]:
train_track_paths = np.array([train_paths[train_track == x][0] for x in train_uni])
test_track_paths = np.array([test_paths[test_track == x][0] for x in test_uni])
test_full_track_paths = np.array([test_full_paths[test_full_track == x][0] for x in test_full_uni])

In [ ]:
evaluator = Evaluator(EXP_DIR, GPU_ID)
traintrain = evaluator.calculate_distances(train_track_emb,train_track_emb)
testtest = evaluator.calculate_distances(test_track_emb,test_track_emb)
testtrain = evaluator.calculate_distances(test_track_emb,train_track_emb)
traintest = evaluator.calculate_distances(train_track_emb,test_track_emb)
querytest = evaluator.calculate_distances(query_emb, test_track_emb)
querytrain = evaluator.calculate_distances(query_emb, train_track_emb)
querytestfull = evaluator.calculate_distances(query_emb, test_full_track_emb)
traintestfull = evaluator.calculate_distances( train_track_emb, test_full_track_emb)

In [ ]:
t = np.argsort(traintestfull,axis = 1)[6,:8]
vis_img_group(test_full_track_paths[t])

In [ ]:
print(np.unique(train_uni_id))
print(traintestfull.shape)
ID = 18
selected_list = np.unique(np.argsort(traintestfull, axis = 1)[:,:12])
print(selected_list)
# vis_img_group(test_full_track_paths[selected_list])
GT = np.array([592,8,648,762,109,727,508,761,768,758,71,483,185,630,260,96,269,79,256,546,259,425,683,166,771,217,658,759,86,283])
correct_list = np.intersect1d(GT,selected_list)
correct = len(correct_list)
incorrect = len(selected_list) - correct
print('Correct', correct, '/', len(GT),' incorrect', incorrect)
# vis_img(train_track_paths[ID])
# vis_img_group(test_full_track_paths[np.argsort(traintestfull[ID,:])],5,5)
incorrect_list = np.logical_not(np.isin(GT,selected_list))
vis_img_group(test_full_track_paths[GT[incorrect_list]],5,5)

In [ ]:
tid = 18
print(test_uni)
print(test_uni[tid])
vis_img(test_track_paths[tid])
# vis_img_group(test_track_paths[np.argsort(testtest[tid,:])],4,5)

In [ ]:
queryId = 1040
print(query_paths[queryId])
vis_img(query_paths[queryId])
vis_img_group(test_track_paths[np.argsort(querytest[queryId,:])],4,5)
# vis_img_group(test_full_track_paths[np.argsort(querytestfull[queryId,:])],4,5)
# vis_img_group(train_track_paths[np.argsort(querytrain[queryId,:])],4,5)

In [ ]:
print(test_uni)
TRACKID = 8
TOPTEST = np.argsort(testtest[TRACKID,:])
TOPTRAIN = np.argsort(testtrain[TRACKID,:])
vis_img(test_track_paths[TRACKID])
print(test_uni[TRACKID],"->", test_uni[TOPTEST])
vis_img_group(test_track_paths[TOPTEST], 5,4)

In [ ]:
vis_img_group(train_track_paths[TOPTRAIN], 5,4)

In [ ]:
# vis_img_group(train_track_paths[TOPTRAIN], 5,4)
top_train = TOPTRAIN[0]
x = train_uni[top_train]
a = train_id == np.unique(train_id[train_track == x])
train_multi_views = np.unique(train_track[a])
ext = np.argwhere(train_multi_views[1] == train_uni)
ext = np.squeeze(ext)
vis_img(train_track_paths[ext])
print(traintest.shape)
test_sec_top = np.argsort(traintest[ext,:])
print(test_uni[test_sec_top])
vis_img_group(test_track_paths[test_sec_top], 5,4)

In [ ]:
track_emb = np.array([np.average(img_emb[imgs_tracklet == x], axis = 0) for x in track_uni])
track_dist = evaluator.calculate_distances(track_emb,track_emb)

# track_bus_type = np.array([np.argmax(np.sum(scores[imgs_tracklet == x], axis = 0)) for x in track_uni])

In [ ]:
PICID = 90
vis_img(img_paths[PICID])
print(imgs_tracklet[PICID])
score_sorted = np.argsort(dist[PICID,:])
print(imgs_tracklet[score_sorted])
vis_img_group(img_paths[score_sorted[:20]],4,5)

In [ ]:
TRACK = 797
vis_img(img_paths[TRACK])
top20 = img_paths[np.argsort(dist[TRACK,:])[:30]]
print(np.argsort(dist[TRACK,:])[:30])
# print(view_dist[np.argsort(dist[TRACK,:])[:30]])
# print(view_cls[TRACK])
vis_img_group(top20,4,5)

In [ ]:
evaluator = Evaluator(EXP_DIR, GPU_ID)
querytrainfull = evalusator.calculate_distances(query_emb,train_full_emb)